In [1]:
import pymysql

# your username and password go here!
creds = {'user': 'fausto', 
         'pswd': 'dapuzzo'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [ ]:
createPagelinks = '''CREATE TABLE IF NOT EXISTS pagelinks (
                page_id INT(8) UNSIGNED NOT NULL,
                pl_count INT(8) UNSIGNED NOT NULL,
                pl_is_epa INT(8) UNSIGNED NOT NULL,
                ratio INT(8) UNSIGNED NOT NULL,
                PRIMARY KEY (page_id)
                ) DEFAULT CHARACTER SET utf8'''

In [ ]:
cur.execute(createPagelinks)

In [11]:
cols_for_db = ["page_id", "pl_count", "pl_is_epa", "ratio"]

In [2]:
import pandas as pd
epa_links = pd.read_csv("../CategoryAndPageLinks/pl_stats_epa.csv")
nepa_links = pd.concat([pd.read_csv("../CategoryAndPageLinks/pl_stats_nepa_batch{}.csv".format(batchNum)) for batchNum in range(1, 9)])

In [3]:
epa_links.columns = ["page_id", "pl_count", "pl_is_epa", "ratio"]
nepa_links.columns = ["page_id", "pl_count", "pl_is_epa", "ratio"]

In [4]:
# convert fraction to per-thousands (%°)
epa_links["ratio"]=epa_links["ratio"]*1000
# convert values to integers
epa_links[["page_id", "pl_count", "pl_is_epa", "ratio"]] = \
epa_links[["page_id", "pl_count", "pl_is_epa", "ratio"]].astype(int)

In [5]:
# convert fraction to per-thousands (%°)
nepa_links["ratio"]=nepa_links["ratio"]*1000
# convert values to integers
nepa_links[["page_id", "pl_count", "pl_is_epa", "ratio"]] = \
nepa_links[["page_id", "pl_count", "pl_is_epa", "ratio"]].astype(int)

In [6]:
import numpy as np
redundant = pd.merge(epa_links, nepa_links, how='inner', left_on="page_id", right_on="page_id")
redundantIDs = redundant['page_id'].tolist()
nepa_links_filtered = nepa_links.loc[~nepa_links['page_id'].isin(redundantIDs)].copy()

In [7]:
master_links = pd.concat([epa_links, nepa_links_filtered])

In [8]:
master_links = master_links.where((pd.notnull(master_links)), None) #converts NaNs to None type, required for mySQL

In [9]:
master_links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83290 entries, 0 to 7612
Data columns (total 4 columns):
page_id      83290 non-null int32
pl_count     83290 non-null int32
pl_is_epa    83290 non-null int32
ratio        83290 non-null int32
dtypes: int32(4)
memory usage: 1.9 MB


In [12]:
ma_for_db = master_links[cols_for_db].to_dict(orient='records') #provides each row as a dict;
#keys are colnames, values are column values, the order of keys is retained from row to row

In [13]:
cur.execute("SET autocommit=0") #makes batch inserts SO MUCH EASIER

0

In [18]:
#generates the template query for insertions into db
insertQ = "INSERT IGNORE INTO pagelinks({}) VALUES ({})"
cols = ",".join(cols_for_db)
val_format = ",".join(['%s'] * len(list(ma_for_db[0].keys())))
formatQ = insertQ.format(cols, val_format)
formatQ

'INSERT IGNORE INTO pagelinks(page_id,pl_count,pl_is_epa,ratio) VALUES (%s,%s,%s,%s)'

In [19]:
print(type(ma_for_db))
ma_for_db[:3]

<class 'list'>


[{'page_id': 25, 'pl_count': 672, 'pl_is_epa': 23, 'ratio': 34},
 {'page_id': 290, 'pl_count': 193, 'pl_is_epa': 10, 'ratio': 51},
 {'page_id': 307, 'pl_count': 1508, 'pl_is_epa': 84, 'ratio': 55}]

In [34]:
for record in ma_for_db:
    values = list(record.values())
    values = map(int, values)
    cur.execute(formatQ,values)

connection.commit()

AttributeError: 'map' object has no attribute 'translate'

In [ ]:
#interestingCols = ["page_id", "pl_count", "pl_is_epa", "ratio_x1000"]
interestingQ = "SELECT * FROM pagelinks"
#colNameString = ",".join(interestingCols)
preppedIQ = interestingQ.format(colNameString)

cur.execute(interestingQ)
rows = cur.fetchall() #returns tuple of tuples, preserving column order

easyPeasy = pd.DataFrame(list(rows), columns=interestingCols)
easyPeasy

In [23]:
cur.execute("SELECT * FROM pagelinks")

0

In [ ]:
cur.close()
connection.close()